In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import splitfolders
from keras.layers import *
from keras.models import *
from keras.callbacks import *

In [4]:
#division des jeu de données
splitfolders.ratio("Datasets", output="dataset", seed=42, ratio=(.7,.2,.1))

Copying files: 20212 files [00:14, 1403.24 files/s]


In [5]:
# generation d'image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.1, zoom_range = 0.1, horizontal_flip = True, rotation_range=15)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
training_set = train_datagen.flow_from_directory("dataset/train",target_size=(64,64),batch_size=32, class_mode="categorical")
testing_set = test_datagen.flow_from_directory("dataset/test",target_size=(64,64),batch_size=32,class_mode="categorical")
validation_set = val_datagen = val_datagen.flow_from_directory("dataset/val",target_size=(64,64),batch_size=32,class_mode="categorical")

Found 14142 images belonging to 10 classes.
Found 2032 images belonging to 10 classes.
Found 4038 images belonging to 10 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt # Nouvelle façon d'importer
import keras

def build_model(hp):
    model = keras.Sequential()
    
    # 1ère couche Conv
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(64, 64, 3)
    ))
    
    # 2ème couche Conv
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ))
    
    # Pooling : Ajout de 2 dans les choix, car 3 ou 5 peut être trop grand pour 64x64
    model.add(keras.layers.MaxPool2D(
        pool_size=hp.Choice('pool_1_size', values=[2, 3])
    ))
    
    # Dropout variable pour trouver le meilleur taux
    model.add(keras.layers.Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))
    
    # 3ème couche Conv
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
        activation='relu'
    ))
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    
    # Couche de sortie (4 classes)
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_epochs=25,          
    factor=5,
    directory='output_new',
    project_name='classification_Hyperband'
)


# Callback pour arrêter l'entraînement si un modèle ne s'améliore pas
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(training_set,epochs = 5, callbacks=[EarlyStopping(monitor='val_loss', mode='min',patience=10)], validation_data=(testing_set))

Trial 35 Complete [00h 04m 47s]
val_accuracy: 0.26279526948928833

Best val_accuracy So Far: 0.711614191532135
Total elapsed time: 09h 09m 47s

Search: Running Trial #36

Value             |Best Value So Far |Hyperparameter
32                |96                |conv_1_filter
3                 |3                 |conv_1_kernel
32                |48                |conv_2_filter
5                 |5                 |conv_2_kernel
3                 |3                 |pool_1_size
0.2               |0.3               |dropout_rate
48                |48                |conv_3_filter
5                 |5                 |conv_3_kernel
96                |64                |dense_1_units
0.0001            |0.001             |learning_rate
5                 |25                |tuner/epochs
0                 |5                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/5
339/442 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/

In [ ]:
tuner.results_summary()

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.save('model.h5')